In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [145]:
#1
file_path = '/content/drive/MyDrive/GYAFC_Corpus/Entertainment_Music/train/informal'   # Replace 'your_text_file.txt' with the path to your text file
with open(file_path, 'r') as file:
    X = file.readlines()

for i  in range(len(X)):
  X[i] = X[i].split("\n")[0]
  X[i] = X[i].lower()

In [146]:
type(X)

list

In [147]:
X = X[40000:46000]

In [148]:
#Adding token 0 to informal sentences
y  = []
for i in range(len(X)):
    y.append(0)

In [149]:
file_path = '/content/drive/MyDrive/GYAFC_Corpus/Entertainment_Music/train/formal'   # Replace 'your_text_file.txt' with the path to your text file
with open(file_path, 'r') as file:
    X_formal = file.readlines()

for i  in range(len(X_formal)):
  X_formal[i] = X_formal[i].split("\n")[0]
  X_formal[i] = X_formal[i].lower()

In [150]:
X_formal = X_formal[40000:46000]

In [151]:
#Adding token 1 to formal sentences
y_formal = []
for i in range(len(X_formal)):
  y_formal.append(1)

In [137]:
dict_formal = {"X" : X_formal,
               "y": y_formal}
dict_informal = {"X": X,
                 "y": y}
import pandas as pd
df_formal = pd.DataFrame(dict_formal)
df_informal = pd.DataFrame(dict_informal)
df = pd.concat([df_formal,df_informal],axis = 0)

In [152]:
# Step 2: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['X'].tolist(), df['y'].tolist(), test_size=0.2, random_state=42)


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Step 2: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['X'].tolist(), df['y'].tolist(), test_size=0.2, random_state=42)

# Step 3: Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenized_inputs_train = tokenizer(X_train, padding=True, truncation=True, return_tensors="pt")
tokenized_inputs_test = tokenizer(X_test, padding=True, truncation=True, return_tensors="pt")

# Step 4: Create PyTorch Dataset
class FormalityDataset(Dataset):
    def __init__(self, tokenized_inputs, labels):
        self.tokenized_inputs = tokenized_inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokenized_inputs["input_ids"][idx],
            "attention_mask": self.tokenized_inputs["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = FormalityDataset(tokenized_inputs_train, y_train)
test_dataset = FormalityDataset(tokenized_inputs_test, y_test)

# Step 5: Fine-Tune the Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

# Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Move input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in batch.items()}
# Fine-tuning loop
model.train()
for epoch in range(3):  # Adjust number of epochs as needed
    for batch in train_dataloader:
        optimizer.zero_grad()

        inputs = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Step 6: Save Model and Tokenizer Configurations
model.save_pretrained("model_configs_fscore")
tokenizer.save_pretrained("tokenizer_configs_fscore")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('tokenizer_configs_fscore/tokenizer_config.json',
 'tokenizer_configs_fscore/special_tokens_map.json',
 'tokenizer_configs_fscore/vocab.txt',
 'tokenizer_configs_fscore/added_tokens.json')

In [155]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Step 1: Load the Model and Tokenizer
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/model_configs_fscore")
tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/tokenizer_configs_fscore")

# Step 2: Tokenization
text = input()
inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")

# Step 3: Prepare Input Tensors
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Move tensors to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
model.to(device)

# Step 4: Forward Pass
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Apply softmax to obtain probabilities
probabilities = torch.softmax(logits, dim=1)

# Extract formality scores
formality_score = probabilities[0][1].item()  # Probability of class 1 (informal)


print("Formality Score:", formality_score)




how are you doing?
Formality Score: 0.8359653353691101


In [142]:
len(X_test)

2400

In [156]:

from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

# Step 4: Create PyTorch Dataset
class FormalityDataset(Dataset):
    def __init__(self, tokenized_inputs, labels):
        self.tokenized_inputs = tokenized_inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokenized_inputs["input_ids"][idx],
            "attention_mask": self.tokenized_inputs["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenized_inputs_test = tokenizer(X_test, padding=True, truncation=True, return_tensors="pt")
test_dataset = FormalityDataset(tokenized_inputs_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=8)


# List to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Evaluate the model on the test dataset
model.eval()
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).tolist()

        # Store true and predicted labels
        true_labels.extend(labels.tolist())
        predicted_labels.extend(predictions)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[ 928  236]
 [ 170 1066]]


In [157]:
from sklearn.metrics import precision_score , recall_score
print("Recall_score:",recall_score(true_labels, predicted_labels))
print("Precision_score:", precision_score(true_labels,predicted_labels))

Recall_score: 0.8624595469255664
Precision_score: 0.8187403993855606
